# ETL Bus Position

In [1]:
import pytz
import os
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import explode, col

In [2]:
# Inicializa a sessão Spark
spark = SparkSession.builder.appName("RawToTrustedBatch").getOrCreate()

# Configura o acesso ao MinIO
hadoop_conf = spark.sparkContext._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", "datalake")
hadoop_conf.set("fs.s3a.secret.key", "datalake")
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

spark

In [3]:
# Defina o caminho de entrada e saída com base na data
fuso_horario_brasilia = pytz.timezone('America/Sao_Paulo')
brasilia_time = datetime.now(fuso_horario_brasilia)
today = brasilia_time.strftime('%Y-%m-%d')

raw_path = f"s3a://raw/busdata/{today}"

routes_trusted_path = f"s3a://trusted/busroutes/{today}/"
positions_trusted_path = f"s3a://trusted/buspositions/{today}/"

In [4]:
# Exibe a lista de arquivos
files_df = spark.read.format("binaryFile").load(raw_path)

# Pega o primeiro arquivo para gerar o df_bus_lines
first_raw_file_path = files_df.collect()[0]["path"]
file_name = os.path.basename(first_raw_file_path).replace(".json", "")
                                                          
df_raw = spark.read.json(first_raw_file_path)

# Explodir o array "l" para acessar as informações de cada linha de ônibus
df_lines = df_raw.select(explode(col("l")).alias("linha"))

# DataFrame com as informações de "c" até "qv" (linhas de ônibus)
df_bus_lines = df_lines.select(
    col("linha.cl").alias("codigo_trajeto"),
    col("linha.sl").alias("sentido"),
    col("linha.c").alias("letreiro"),
    col("linha.lt0").alias("terminal_primario"),
    col("linha.lt1").alias("terminal_secundario"),
    col("linha.qv").alias("qnt_veiculos")
)
# Salva o df_bus_lines no caminho apenas uma vez
# Define o caminho de destino usando o nome do arquivo original
destination_path = os.path.join(routes_trusted_path, file_name)
    
# Salva o DataFrame processado no formato Parquet
#df_bus_lines.write.mode("overwrite").parquet(destination_path)
df_bus_lines.write.mode("overwrite").json(routes_trusted_path)
df_bus_lines.show()

+--------------+-------+--------+-----------------+--------------------+------------+
|codigo_trajeto|sentido|letreiro|terminal_primario| terminal_secundario|qnt_veiculos|
+--------------+-------+--------+-----------------+--------------------+------------+
|         32999|      2| 273V-10|      METRÔ PENHA|PARADA XV DE NOVE...|           3|
|         32991|      2| 273G-10|METRÔ ARTUR ALVIM|          JD. HELENA|           6|
|         33920|      2| 6063-10|   TERM. VARGINHA|        JD. VARGINHA|           8|
|           145|      1| 2765-10|    METRÔ TATUAPÉ|          VL. CISPER|           5|
|         33812|      2| 3745-10|    METRÔ TATUAPÉ|       JD. IMPERADOR|           6|
|         33911|      2| 6051-10|   TERM. VARGINHA|          JD. ITAJAÍ|           5|
|           516|      1| 106A-10|       ITAIM BIBI|       METRÔ SANTANA|           6|
|          1655|      1| 513L-10|        PQ. BELÉM|        TERM. SACOMÃ|           2|
|           543|      1| 1743-10|          SHOP. D|   

In [5]:
# Itera sobre os arquivos para processar as posições dos veículos
for row in files_df.collect():
    raw_file_path = row["path"]
    raw_file_date = raw_file_path.split('_')[-1].split('.')[0]
    df_raw = spark.read.json(raw_file_path)
    
    # Explodir o array "l" para acessar as informações de cada linha de ônibus
    df_lines = df_raw.select(explode(col("l")).alias("linha"))

    # Explodir o array "vs" para acessar as informações de cada veículo dentro da linha de ônibus
    df_vehicles = df_lines.select(
        col("linha.cl").alias("codigo_trajeto"),
        col("linha.sl").alias("sentido"),
        explode(col("linha.vs")).alias("vehicle")
    )

    # DataFrame com as informações de "p" até "px" (posições dos veículos)
    df_vehicles_position = df_vehicles.select(
        col("codigo_trajeto"),
        col("sentido"),
        col("vehicle.p").alias("prefixo_veiculo"),
        col("vehicle.py").alias("latitude"),
        col("vehicle.px").alias("longitude")
    )
    
    # Salva as posições dos veículos no caminho de saída
    df_vehicles_position.write.mode("overwrite").json(positions_trusted_path + 'positions_' + raw_file_date)

In [6]:
df_vehicles_position.show()

+--------------+-------+---------------+-------------------+-------------------+
|codigo_trajeto|sentido|prefixo_veiculo|           latitude|          longitude|
+--------------+-------+---------------+-------------------+-------------------+
|         32999|      2|          32007|         -23.521937|        -46.4241185|
|         32999|      2|          32050|-23.521850999999998|        -46.4244515|
|         32999|      2|          32004|        -23.5331465|         -46.543474|
|         32999|      2|          32638|        -23.5231385|        -46.5208425|
|         32999|      2|          32009|        -23.5331465|         -46.543474|
|         33920|      2|          66298|         -23.783673|        -46.6827105|
|         33920|      2|          66275|         -23.783673|        -46.6827105|
|         33920|      2|          66271|         -23.783673|        -46.6827105|
|         33920|      2|          66354|         -23.783673|        -46.6827105|
|         33920|      2|    

In [7]:
# Encerra a sessão Spark
spark.stop()

In [6]:
# ----------------------------------------------------------------------------------------------------------------- #